In [3]:
# Import necessary libraries
import os
from pathlib import Path
import sys  # Keep for potential error output

# --- Core Logic (Mostly unchanged, includes previous refinements) ---

def generate_folder_structure(directory_path, indent="", is_last=True, max_depth=None,
                             current_depth=0, ignore_patterns=None, max_files=None,
                             processed_paths=None): # Add processed_paths to prevent infinite loops
    """
    Recursively generate a tree-like folder structure visualization. Handles symlinks.
    """
    # Initialize processed_paths set on the first call
    if processed_paths is None:
        processed_paths = set()

    try:
        # Resolve symlinks and get real path, but handle broken links gracefully
        # Use stat() to check existence without following the link immediately if it might loop
        if directory_path.is_symlink():
             real_path = directory_path.resolve()
             # Check for loops
             if real_path in processed_paths:
                 print(f"{indent}├── {directory_path.name} -> [Symbolic link loop detected]")
                 return
             # Check if resolved path exists
             if not real_path.exists():
                  print(f"{indent}├── {directory_path.name} -> [Broken symbolic link]")
                  return
             # Add the resolved path to prevent loops in this branch
             processed_paths.add(real_path)
             # Continue processing with the resolved path but display the link name
             path_to_process = real_path
             display_name = f"{directory_path.name} -> {path_to_process.name}" # Show link target
        else:
             path_to_process = directory_path
             display_name = path_to_process.name
             # Add the canonical path to prevent loops if entered via different relative paths
             processed_paths.add(path_to_process.resolve())

        # Depth check
        if max_depth is not None and current_depth > max_depth:
            return

        # Skip if matches ignore pattern
        if ignore_patterns and isinstance(ignore_patterns, (list, tuple)):
            path_str = str(path_to_process.resolve())
            if any(pattern in path_str for pattern in ignore_patterns):
                return

        # Print the current directory/file
        connector = "└── " if is_last else "├── "
        print(f"{indent}{connector}{display_name}", end="")

        # Add comment for file types if needed
        if path_to_process.is_file():
            extension = path_to_process.suffix.lower()
            comment = ""
            # --- Add more file type comments as needed ---
            if extension == ".py": comment = "# Python script"
            elif extension == ".ipynb": comment = "# Jupyter Notebook"
            elif extension == ".css": comment = "# CSS styles"
            elif extension == ".js": comment = "# JavaScript code"
            elif extension == ".html": comment = "# HTML template"
            elif extension == ".json": comment = "# JSON data"
            elif extension == ".md": comment = "# Markdown file"
            elif extension == ".txt": comment = "# Text file"
            elif extension == ".csv": comment = "# CSV data"
            elif extension == ".pkl": comment = "# Pickle file"
            elif extension == ".h5": comment = "# HDF5 file"
            elif extension == ".env": comment = "# Environment variables"
            # --- End file type comments ---
            print(f"  {comment}" if comment else "")
        else:
            print() # Just a newline for directories

        # If it's a directory, process its contents
        if path_to_process.is_dir() and (max_depth is None or current_depth < max_depth):
            items = []
            try:
                # Get all items, handling potential permission errors
                items = list(path_to_process.iterdir())
            except PermissionError:
                new_indent = indent + ("    " if is_last else "│   ")
                print(f"{new_indent}└── [Permission Denied]")
                return
            except OSError as e:
                new_indent = indent + ("    " if is_last else "│   ")
                print(f"{new_indent}└── [Error reading directory: {e}]")
                return

            # Filter out ignored items *before* sorting and limiting
            if ignore_patterns and isinstance(ignore_patterns, (list, tuple)):
                 filtered_items = [
                    item for item in items
                    if not any(pattern in str(item.resolve()) for pattern in ignore_patterns)
                 ]
            else:
                 filtered_items = items # No filtering needed

            # Separate directories and files from the filtered list
            # Sort alphabetically by name
            dirs = sorted([item for item in filtered_items if item.is_dir()], key=lambda p: p.name)
            files = sorted([item for item in filtered_items if item.is_file()], key=lambda p: p.name)
            # Handle other types like symlinks (if not filtered) separately or group with files
            others = sorted([item for item in filtered_items if not item.is_dir() and not item.is_file()], key=lambda p: p.name)
            files.extend(others) # Treat symlinks etc. like files for display order/limits

            # Apply file limit if specified
            original_file_count = len(files)
            if max_files is not None and original_file_count > max_files:
                files_to_show = files[:max_files]
                has_hidden_files = True
                hidden_count = original_file_count - max_files
            else:
                files_to_show = files
                has_hidden_files = False
                hidden_count = 0

            # Combine sorted directories and limited/sorted files
            sorted_items_to_process = dirs + files_to_show

            # Process each item
            num_items = len(sorted_items_to_process)
            for i, item in enumerate(sorted_items_to_process):
                is_item_last = (i == num_items - 1) and not has_hidden_files
                new_indent = indent + ("    " if is_last else "│   ")

                # Recursively process this item, passing the processed_paths set
                generate_folder_structure(
                    item,
                    new_indent,
                    is_item_last,
                    max_depth,
                    current_depth + 1,
                    ignore_patterns,
                    max_files,
                    processed_paths.copy() # Pass a copy to handle separate branches correctly
                )

            # Add indicator for hidden files if needed
            if has_hidden_files:
                 new_indent = indent + ("    " if is_last else "│   ")
                 print(f"{new_indent}└── ... ({hidden_count} more file(s) not shown)")

    except Exception as e:
        # Catch potential errors during path processing (e.g., race conditions, weird file issues)
        connector = "└── " if is_last else "├── "
        try:
            # Try to print the name even if processing failed
            print(f"{indent}{connector}{directory_path.name}  [Error processing: {e}]")
        except: # Fallback if getting name fails
             print(f"{indent}{connector}[Error processing item: {e}]")


# --- Jupyter-Friendly Wrapper Function ---

def display_folder_tree(target_path, max_depth=None, max_files=None, ignore_patterns=None):
    """
    Sets up and calls the recursive function to display the folder tree.

    Args:
        target_path (str): The path to the directory you want to visualize.
        max_depth (int, optional): Maximum depth to traverse. Defaults to None (unlimited).
        max_files (int, optional): Maximum number of files to show per directory. Defaults to None (unlimited).
        ignore_patterns (list, optional): List of strings/patterns to ignore in paths
                                           (e.g., [".git", "__pycache__", ".ipynb_checkpoints"]).
                                           Defaults to None.
    """
    try:
        # Convert string path to Path object
        start_path = Path(target_path)

        if not start_path.exists():
            print(f"Error: Starting path '{target_path}' does not exist.", file=sys.stderr)
            return

        # Resolve to absolute path for clarity in the output, but handle potential errors
        try:
            resolved_path = start_path.resolve()
        except OSError as e:
            print(f"Warning: Could not resolve path '{target_path}'. Using original. Error: {e}", file=sys.stderr)
            resolved_path = start_path # Use the original path if resolve fails

        print(f"Folder Tree for: {resolved_path}") # Print the root directory clearly

        # Start the recursive generation
        generate_folder_structure(
            resolved_path,     # Start with the resolved path object
            indent="",         # Start with no indent
            is_last=True,      # The root is always the 'last' at its level conceptually
            max_depth=max_depth,
            current_depth=0,   # Start at depth 0
            ignore_patterns=ignore_patterns,
            max_files=max_files,
            processed_paths=None # Initialize loop detection
        )

    except Exception as e:
        print(f"\nAn unexpected error occurred during setup: {e}", file=sys.stderr)

# ==================================================================
# --- How to Use in Jupyter ---
# ==================================================================

# 1. !!! EDIT THIS LINE to your desired folder path !!!
#    Examples:
#    my_folder_path = "."                       # Current directory where the notebook is running
#    my_folder_path = ".."                      # Parent directory
#    my_folder_path = "/path/to/your/project"   # Linux/macOS absolute path
#    my_folder_path = "C:/Users/YourUser/Documents" # Windows absolute path (use forward slashes)
#    my_folder_path = r"C:\Users\YourUser\Documents" # Windows absolute path (raw string)


In [4]:

my_folder_path = r"D:\OneDrive - Green Energy\Sakib\E2-F5-TTS"  # <-- CHANGE THIS PATH

# 2. Define optional parameters (uncomment and modify lines below if needed)
my_max_depth = 3       # Limit recursion depth (e.g., show only 3 levels deep)
my_max_files = 10      # Show max 10 files per folder (None means show all)
my_ignore = [          # List of directory/file name patterns to exclude
    ".git",
    "__pycache__",
    ".ipynb_checkpoints",
    "node_modules",
    ".vscode",
    "build",
    "dist",
    ".env"             # Add any other patterns you want to ignore
]

# 3. !!! RUN ONE of the following calls !!!
#    (Uncomment the line you want to use)

# --- Option A: Run with just the path ---
# display_folder_tree(my_folder_path)

# --- Option B: Run with path and all options ---
display_folder_tree(
    my_folder_path,
    max_depth=my_max_depth,
    max_files=my_max_files,
    ignore_patterns=my_ignore
)

# --- Option C: Run with only some options (Example: path and depth) ---
# display_folder_tree(
#     my_folder_path,
#     max_depth=my_max_depth
# )

# --- Option D: Run with path and ignore patterns ---
# display_folder_tree(
#     my_folder_path,
#     ignore_patterns=my_ignore
# )

Folder Tree for: D:\OneDrive - Green Energy\Sakib\E2-F5-TTS
└── E2-F5-TTS
    ├── backup
    │   ├── ckpts
    │   │   └── README.md  # Markdown file
    │   ├── data
    │   │   ├── Emilia_ZH_EN_pinyin
    │   │   ├── .DS_Store
    │   │   └── librispeech_pc_test_clean_cross_sentence.lst
    │   ├── model
    │   │   ├── backbones
    │   │   ├── __init__.py  # Python script
    │   │   ├── cfm.py  # Python script
    │   │   ├── dataset.py  # Python script
    │   │   ├── ecapa_tdnn.py  # Python script
    │   │   ├── modules.py  # Python script
    │   │   ├── trainer.py  # Python script
    │   │   ├── utils.py  # Python script
    │   │   └── utils_infer.py  # Python script
    │   ├── samples
    │   │   ├── country.flac
    │   │   ├── main.flac
    │   │   ├── story.toml
    │   │   ├── story.txt  # Text file
    │   │   └── town.flac
    │   ├── scripts
    │   │   ├── count_max_epoch.py  # Python script
    │   │   ├── count_params_gflops.py  # Python script
    │   │   ├── e